In [1]:
import mlflow
import mlflow.keras
import tensorflow as tf
from tensorflow import keras
import os
from os import path, listdir


In [2]:
def load_image_dataset(file_path):
    all_image_dirs = [os.path.join(file_path, f) for f in os.listdir(file_path) if not os.path.isdir(os.path.join(file_path, f))]
    all_image_labels = []
    for f in all_image_dirs:
        if "cat" in f:
            all_image_labels.append(0)
        else:
            all_image_labels.append(1)
    return all_image_dirs, all_image_labels

# Provide the correct path to your dataset
train_path = r"C:/Users/LENOVO/Desktop/mlflowshit/data/train"
all_image_dirs, all_image_labels = load_image_dataset(train_path)


In [3]:
num_train_image = int(len(all_image_labels) * 0.8 // 1)
train_image_dirs, train_label = all_image_dirs[:num_train_image], all_image_labels[:num_train_image]
test_image_dirs, test_label = all_image_dirs[num_train_image:], all_image_labels[num_train_image:]

train_path_label = tf.data.Dataset.from_tensor_slices((train_image_dirs, train_label))
test_path_label = tf.data.Dataset.from_tensor_slices((test_image_dirs, test_label))


In [4]:
def load_and_preprocess_image(path):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [192, 192])
    image /= 255.0  # Normalization to [0,1]
    image = 2 * image - 1  # Normalize to [-1,1]
    return image

train_image_label_ds = train_path_label.map(lambda x, y: (load_and_preprocess_image(x), y))
test_image_label_ds = test_path_label.map(lambda x, y: (load_and_preprocess_image(x), y)).batch(1)


In [5]:
# Load pre-trained MobileNetV2 and add layers on top
mobile_net = tf.keras.applications.MobileNetV2(input_shape=(192, 192, 3), include_top=False)
mobile_net.trainable = False  # Freeze the base model

# Build a custom Sequential model
cnn_model = keras.models.Sequential([
    mobile_net,
    keras.layers.GlobalAveragePooling2D(),
    keras.layers.Flatten(),
    keras.layers.Dense(64, activation="relu"),
    keras.layers.Dense(2, activation="softmax")
])

cnn_model.compile(optimizer=tf.keras.optimizers.Adam(),
                  loss='sparse_categorical_crossentropy',
                  metrics=["accuracy"])


In [6]:
BATCH_SIZE = 32
EPOCHS = 2
AUTOTUNE = tf.data.experimental.AUTOTUNE
steps_per_epoch = len(train_image_dirs) // BATCH_SIZE

train_ds = train_image_label_ds.shuffle(buffer_size=len(train_image_dirs))
train_ds = train_ds.repeat()
train_ds = train_ds.batch(BATCH_SIZE)
train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)


In [7]:
learning_rates = [0.001, 0.0001]  #  learning rates to experiment with

for lr in learning_rates:
    with mlflow.start_run():
        # Log parameters for the current run
        mlflow.log_param("batch_size", BATCH_SIZE)
        mlflow.log_param("epochs", EPOCHS)
        mlflow.log_param("learning_rate", lr)

        # Compile model with the new learning rate
        cnn_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
                          loss='sparse_categorical_crossentropy',
                          metrics=["accuracy"])

        # Train the model
        history = cnn_model.fit(train_ds, epochs=EPOCHS, steps_per_epoch=steps_per_epoch)

        # Log accuracy and loss metrics
        mlflow.log_metric("accuracy", history.history['accuracy'][-1])
        mlflow.log_metric("loss", history.history['loss'][-1])

        # Log the model in MLflow
        mlflow.keras.log_model(cnn_model, "model")


Epoch 1/2
625/625 ━━━━━━━━━━━━━━━━━━━━ 471s 588ms/step - accuracy: 0.9771 - loss: 0.0672
Epoch 2/2
625/625 ━━━━━━━━━━━━━━━━━━━━ 436s 698ms/step - accuracy: 0.9903 - loss: 0.0282


2024/10/28 01:15:54 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
2024/10/28 01:17:25 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Epoch 1/2
625/625 ━━━━━━━━━━━━━━━━━━━━ 561s 611ms/step - accuracy: 0.9953 - loss: 0.0149
Epoch 2/2
625/625 ━━━━━━━━━━━━━━━━━━━━ 413s 660ms/step - accuracy: 0.9966 - loss: 0.0105


2024/10/28 01:33:41 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
2024/10/28 01:34:38 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


In [9]:
# Query the runs and sort by accuracy to get the best model
best_run = mlflow.search_runs(order_by=["metrics.accuracy DESC"]).iloc[0]

# Get the URI of the best model
best_model_uri = best_run.artifact_uri + "/model"

# Load the best model
best_model = mlflow.keras.load_model(best_model_uri)

# You can now use `best_model` for making predictions or further testing
